In [ ]:
library(Seurat)
library(data.table)
library(ggplot2)
library(readxl)
library(plyr)

library(dplyr)
library(extrafont)
library(reshape2)

library(viridis)
library(knitr)
library(stringr)
library(NMF)
library(rsvd)
library(lme4)
library(RColorBrewer)

library(gtools)
library(gprofiler2)


#font_import()
datasets <- readRDS(".../RData/SEZ_seurat_object.rds")
DefaultAssay(datasets) <- "RNA"
levels(datasets)

In [ ]:
library(Seurat)
library(data.table)
library(ggplot2)

library(dplyr)
library(plyr)
library(readxl)

In [ ]:
setwd(".../SVZ/Test_Code")

In [ ]:
#y subset for astrocytes/NSCs and neurons
NG<-subset(datasets,ident=c('16','0','7','8',
                            
                            '19',
                            '11','13','14',
                            '2','3','12','10',
                            '9'
                            
                            ))#,'15''18''6','17','1','5','4',
droplevels(NG)
levels(NG)

In [ ]:
ucols_NG<-c("#993399", "#FFCCFF", '#CC0099' ,
             "#999999",
          
             "#333333",'#3333FF',"#33CCFF","#99FFFF",#"#9933FF",'#66CC99',"#009966","#006633",
             '#99FF33', "#999900", "#CCCC66", "#666633", 
             "#9966CC") #

In [ ]:
#csv containing the marker genes of each reference cell type
markers <- read.csv(file = ".../RData/Zywitza_and_Xie_ds2_terms.csv", header = T, stringsAsFactors = F)
markers[markers==""]<-NA
head(markers)

In [ ]:
#eliminate mitocondrial genes from the reference
selected_terms<-markers 
#selected_terms_2
selected_terms_2<-selected_terms
for(i in 1:ncol(selected_terms_2)){
    df_i<-selected_terms_2[i]
    for(x in 1:nrow(df_i)){
        val<-df_i[x,1]
        selected_terms_2[x,i]<-ifelse(grepl("^MT-", val),"NA",val)
    }
    
    
    }
#selected_terms_2#no MT-genes
#selected_terms_2<-selected_terms_2[2:16]
selected_terms_2

In [ ]:
#calculate the module scores
start<-length(NG@meta.data)
for (column in 1:ncol(selected_terms_2)){
    NG <- AddModuleScore(NG, features = list(na.omit(selected_terms_2[,column])), name = colnames(selected_terms_2)[column])
    }

In [ ]:
start
a=start+1
a
l=length(NG@meta.data)[]
assays<-colnames(NG@meta.data[,a:l])
assays
NG_2<-NG
list_of_average<-list()
meta<-NG_2@meta.data

for (i in 1:length(assays)){
    ass_i<-assays[i]
    vec<-c('cluster', ass_i)
    
    data<-meta[,vec]
    

    data_2<-data %>%
    group_by(cluster) %>%
    summarise_at(vars(ass_i), list(name = mean))
    list_of_average[[ass_i]]<-data_2
    
    
}
#list_of_average_2
clusters_vec<-list_of_average[[1]]$cluster
list_of_average_2<-list_of_average
list_of_average_2<-lapply(list_of_average_2, "[", -c(1))
list_of_average_2_df<-do.call("cbind", list_of_average_2)

colnames(list_of_average_2_df)<-assays
rownames(list_of_average_2_df)<-clusters_vec
list_of_average_2_df$cluster<-rownames(list_of_average_2_df)
data1 <- melt(list_of_average_2_df)
min<-min(data1$value)
max<-max(data1$value)
min
max

In [ ]:
#you can save the info
write.csv(data1,"data_heatmap_zywitza_xieds2.csv")

In [ ]:
data1=read.csv("data_heatmap_zywitza_xieds2.csv")
head(data1)

In [ ]:
levels(factor(data1$variable))

In [ ]:
#you can skip this or make it nicer, it is just to eliminate the "1"
#that appears in the name after the addmodulescores
order=c(
       'Zywitza_TAPs1','Zywitza_aNSC1','Zywitza_Neuroblasts1','Zywitza_Ependymal1',
       'Zywitza_nicheAstrocytes1','Zywitza_D1_MSNs1','Zywitza_D2_MSN1','Xie_H01','Xie_H11','Xie_H21','Xie_H31','Xie_L01','Xie_L11','Xie_L21','Xie_N41')
no_1=c(
       'Zywitza_TAPs','Zywitza_aNSC','Zywitza_Neuroblasts','Zywitza_Ependymal',
       'Zywitza_nicheAstrocytes','Zywitza_D1_MSNs','Zywitza_D2_MSN','Xie_H0','Xie_H1',
    'Xie_H2','Xie_H3','Xie_L0','Xie_L1','Xie_L2','Xie_N4')

In [ ]:
data1$variable=factor(data1$variable, order)

In [ ]:
levels(data1$variable)=no_1

In [ ]:
cols = brewer.pal(9, "PuRd")#YlOrRd rev
cols

In [ ]:

p<-ggplot(data1, aes(x = factor(cluster,levels=c('16','0','7','8',
                             '19','11','13','14','2','3','12','10','9')), y = variable, fill = value)) +
  geom_tile(color = "black")+
    scale_fill_gradient(low="#E7E1EF", high="#67001F", limits=c(0.25,max),na.value = "white")+# you can change the low limit. In this case, enrichment below 0.25 are depicted in white
 theme(strip.text = element_text(size=15),
          panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
    panel.background = element_blank(), axis.line = element_line(colour = "white"),axis.text=element_text(size=15,color="black"),
          text=element_text( face = "bold" ,family="Arial", size=15),strip.background = element_blank(), axis.text.x = element_text(angle = 45, vjust = 0.5, hjust=0.5,size=15),
        axis.text.y = element_text(size=12),
         axis.title.x = element_blank(),axis.ticks.x=element_blank(),axis.ticks.y=element_blank(),
          axis.title.y = element_blank())
p
tiff("heatmap_zywitza_xieds2.tiff", res = 400, units = "in", width = 7, height = 5)
p
dev.off()